In [ ]:
import socket

hex_data = "ed740b550024effd70726f6772616d6d696e6720696e20707974686f6e2069732066756e"

udp_packet = bytes.fromhex(hex_data)

# Parsowanie nagłówka UDP
src_port = int.from_bytes(udp_packet[0:2], byteorder='big')
dst_port = int.from_bytes(udp_packet[2:4], byteorder='big')
length = int.from_bytes(udp_packet[4:6], byteorder='big')

# Pobranie danych
header_length = 8  # Nagłówek UDP ma zawsze 8 bajtów
data_bytes = udp_packet[header_length:length]  # Pobranie danych
data_str = data_bytes.decode(errors='ignore')  # Konwersja na string

message = f"zad14odpsrc;{src_port};dst;{dst_port};data;{data_str}"

# Wysyłanie do serwera
server_address = ("127.0.0.1", 2910)  # Jeśli serwer działa lokalnie
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.settimeout(5)  # Ustawienie timeoutu na odpowiedź

try:
    sock.sendto(message.encode(), server_address)

    response, _ = sock.recvfrom(1024)
    print("Odpowiedź serwera:", response.decode())
except socket.timeout:
    print("Brak odpowiedzi od serwera.")
finally:
    sock.close()

Wiadomość do wysłania: zad14odp;src;60788;dst;2901;data;programming in python is fun
Odpowiedź serwera: BAD_SYNTAX


In [ ]:
import socket
# Poniżej znajduje się pełny zapis segmentu TCP w postaci szesnastkowej (pole opcji ma 12 bajtów).
hex_data = "0b54898b1f9a18ecbbb164f2801800e3677100000101080a02c1a4ee001a4cee68656c6c6f203a29"

tcp_packet = bytes.fromhex(hex_data)

src_port = int.from_bytes(tcp_packet[0:2], byteorder='big')  # 2 bajty: port źródłowy
dst_port = int.from_bytes(tcp_packet[2:4], byteorder='big')  # 2 bajty: port docelowy

data_offset = (tcp_packet[12] >> 4) * 4  
options_length = data_offset - 20 

data_length = len(tcp_packet) - data_offset

print(f"data_offset: {data_offset},  data_length: {data_length}")
# Pobranie danych
data_bytes = tcp_packet[data_offset:]  # Odczytanie danych
data_str = data_bytes.decode(errors='ignore')  # Konwersja na string

# Formatowanie wiadomości
message = f"zad13odpsrc;{src_port};dst;{dst_port};data;{data_str}"
print("Wiadomość do wysłania:", message)

# Wysyłanie do serwera
server_address = ("127.0.0.1", 2910)  
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.settimeout(5)  

try:
    sock.sendto(message.encode(), server_address)

    # Odbieranie odpowiedzi
    response, _ = sock.recvfrom(1024)
    print("Odpowiedź serwera:", response.decode())
except socket.timeout:
    print("Brak odpowiedzi od serwera.")
finally:
    sock.close()

data_offset: 32,  data_length: 8
Wiadomość do wysłania: zad13odp;src;2900;dst;35211;data;hello :)
Odpowiedź serwera: BAD_SYNTAX


In [ ]:
# Napisz program serwera, który działając pod adresem 127.0.0.1 oraz na określonym porcie UDP, dla pod-
# łączającego się klienta, odbierze od niego wiadomość o treści podanej w zadaniu nr 13 z laboratorium nr 3,
# a następnie odeśle klientowi odpowiedź TAK lub NIE. W przypadku błędnego sformatowania wiadomości,
# serwer odeśle klientowi odpowiedź BAD SYNTAX.

import socket

HOST = '127.0.0.1'
PORT = 2911

def validate_message(message):
    """
    Validates the format of the received message.
    Returns "TAK" if valid, "NIE" if invalid, and "BAD SYNTAX" if the format is incorrect.
    """
    if message.startswith("zad15odpA;"):
        # Validate the structure of message A
        parts = message.split(";")
        if len(parts) == 7 and parts[1] == "ver" and parts[3] == "srcip" and parts[5] == "dstip":
            return "TAK"
        else:
            return "BAD SYNTAX"
    elif message.startswith("zad15odpB;"):
        # Validate the structure of message B
        parts = message.split(";")
        if len(parts) == 7 and parts[1] == "srcport" and parts[3] == "dstport" and parts[5] == "data":
            return "TAK"
        else:
            return "BAD SYNTAX"
    else:
        return "NIE"

# Create a UDP server socket
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as server_socket:
    server_socket.bind((HOST, PORT))
    print(f"Serwer nasłuchuje na {HOST}:{PORT}...")
    
    while True:
        data, addr = server_socket.recvfrom(1024)  # Receive data from client
        message = data.decode()
        print(f"Otrzymano wiadomość od {addr}: {message}")
        
        # Validate the message and prepare a response
        response = validate_message(message)
        print(f"Odpowiedź dla {addr}: {response}")
        
        # Send the response back to the client
        server_socket.sendto(response.encode(), addr)

Odpowiedź serwera A: BAD SYNTAX


In [4]:
import socket

def send_message(server_ip, server_port, message):
    """
    Sends a message to the server and receives a response.
    """
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(5)  # Set a timeout for the response
    try:
        print(f"Sending message: {message}")
        sock.sendto(message.encode(), (server_ip, server_port))
        response, _ = sock.recvfrom(1024)
        print(f"Server response: {response.decode()}")
    except socket.timeout:
        print("No response from server.")
    finally:
        sock.close()

# Server details
server_ip = "127.0.0.1"
server_port = 2911

# Test messages
messageA = "zad15odpA;ver;4;srcip;127.0.0.1;dstip;192.168.0.1;type;6"
messageB = "zad15odpB;srcport;12345;dstport;80;data;Hello, world!"
invalid_message = "invalid;message;format"

# Send test messages
send_message(server_ip, server_port, messageA)  # Should return "TAK"
send_message(server_ip, server_port, messageB)  # Should return "TAK"
send_message(server_ip, server_port, invalid_message)  # Should return "NIE"

Sending message: zad15odpA;ver;4;srcip;127.0.0.1;dstip;192.168.0.1;type;6
Server response: BAD SYNTAX
Sending message: zad15odpB;srcport;12345;dstport;80;data;Hello, world!
Server response: TAK
Sending message: invalid;message;format
Server response: NIE
